# ST IT Cloud - Data and Analytics Test LV.4

Esse teste deve avaliar alguns conceitos de big data e a qualidade técnica na manipulacão de dados, otimização de performance, trabalho com arquivos grandes e tratamento de qualidade.

## Passo a passo

- *Parte teórica:* responda as questões abaixo preenchendo as células em branco.
- *Parte prática:* disponibilizamos aqui 2 cases para, leia os enunciados dos problemas, desenvolver os programas, utilizando a **stack definida durante o processo seletivo**, para entregar os dados de acordo com os requisitos descritos abaixo.



**Faz parte dos critérios de avaliacão a pontualidade da entrega. Implemente até onde for possível dentro do prazo acordado.**

**Os dados de pessoas foram gerados de forma aleatória, utilizando a biblioteca FakerJS, FakerJS-BR e Faker**

LEMBRE-SE: A entrega deve conter TODOS os passos para o avaliador executar o programa (keep it simple).


**Questão 1** - Descreva de forma detalhada quais são as etapas na construção de um pipeline de dados, sem considerar ferramentas específicas, imagine que é seu primeiro contato com o cliente e você precisa entender a demanda dele e explicar quais são os passos que você terá que implementar para entregar a demanda.

In [ ]:
# Resposta: O primeiro passo é entender a demanda fazendo o levantamento dos requisitos. Conhecendo o problema, 
# teremos condições de realizar o desenho da soluçao técnica e planejar o desenvolvimento, homologar e implantar a demanda.

# A construção do pipeline de dados abrange as seguintes etapas: mapeamento dos dados na origem, ingestão do dado bruto no destino (exemplo: camada row data no data lake),
# particionamento (exemplo: parquet), transformação dos dados aplicando as regras de negócio, padronização e limpeza (exemplo:  EMR) e disponibilização da informação em um DW (exemplo: Redshift)



**Questão 2** - Defina com suas palavras um processamento em streaming e processamento em batch. Qual sua experiência com cada uma delas.

In [ ]:
# O processamento em streaming deve ser continuo, pois existe a necessidade de utilizar os dados em tempo real para gerar valor ao negócio. Por exemplo, o cliente ao efetivar uma compra ele já recebe uma pesquisa de satisfação em seu e-mail.
# O processamento em batch pode ter uma janela de processamento programada. Um exemplo para esse modelo são os paineis de indicadores executivos, onde não há necessidade de acompanhando em tempo real. 


**Questão 3** - Quais são as camadas de um Data Lake?

In [ ]:
# As camadas são: 
#    1.Row Data - Armazenamento de dados Brutos 
#    2.Trusted  - Camada onde os dados são desnormalizados e centralizados, aplicando rotinas para limpar, padronizar e definir nomenclaturas amigáveis dos campos.
#    3.Refined  - Camada onde os dados são transformados ou enriquecidos de acordo a regra do negócio

**Questão 4** - Quais as diferenças de um Data Lake e um DW?

In [ ]:
# O Data Lake armazena dados estruturados, semi-estruturados e nao estruturados, que pode vir de diversas fontes (sistemas legados, dados abertos, imagens, redes sociais etc)
# O DW armazena os dados transformados e estruturados e estão prontos para responder as necessidades do négocio.

**Questão 5** - O que é arquitetura Lambda e Kappa? Descreva com suas palavras.

In [ ]:
# Arquitetura Lambda consiste em processar o dado em lote e disponibilizar para uso. Já a arquitetura Kappa não tem a camada de lote, sendo assim o processamento é contínuo.

**Questão 6** - O que é Data Quality para você e como você implementa isso nos seus processos?

**Questão 7** - Em uma escala de 0 a 10, qual seria seu nível de experiência com PySpark?

In [ ]:
4

**Questão 8** - Em uma escala de 0 a 10, qual seria seu nível de experiência com SQL?

In [ ]:
10

**Questão 9** - Descreva suas expeciências com banco de dados SQL e NoSQL.

In [ ]:
# SQL: Trabalho com SQL (T-SQL e Oracle) em toda a minha trajetória profissional, desenvolvendo procedures e querys. Já os banco não relacionais, atuo a 2 anos no DymanoDB e fiz alguns trabalhos acadêmicos utilizando o Neo4j.

**Questão 10** - Tem experiência com versionamento de código? Com quais ferramentas já trabalhou? Descreva.

In [ ]:
# Sim, recentemente utilizo o Git. Porém já trabalhei com o TFS (Team Foundation Service) 

**Questão 11** - Tem experiência em desenvolvimento em cloud? Se sim, especifique a(s) plataforma(s) que já trabalhou e suas principais implementações e conhecimentos em cada serviço.

In [ ]:
# Sim, tenho dois anos de experiencia em desenvolvimento nos serviços da AWS.
# São eles: EMR, Lambda e Step Function.

**Questão 12** - Tem experiência com metodologia ágil? Qual?

In [ ]:
# Sim, Scrum.

In [15]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [16]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')



In [43]:
from pyspark.sql import SparkSession, types
from pyspark.sql import Window, DataFrame, functions
from pyspark.sql.functions import *



# TESTE PRÁTICO

**Problema 1**: Você está recebendo o arquivo 'dados_cadastrais_fake.csv' que contem dados cadastrais de clientes, mas para que análises ou relatórios sejam feitos é necessário limpar e normalizar os dados. Além disso, existe uma coluna com o número de cpf e outra com cnpj, você precisará padronizar deixando apenas dígitos em formato string (sem caracteres especiais), implementar uma forma de verificar se tais documentos são válidos sendo que a informação deve se adicionada ao dataframe em outras duas novas colunas.

Após a normalização, gere reports que respondam as seguintes perguntas:
- Quantos clientes temos nessa base?
- Qual a média de idade dos clientes?
- Quantos clientes nessa base pertencem a cada estado?
- Quantos CPFs válidos e inválidos foram encontrados?
- Quantos CNPJs válidos e inválidos foram encontrados?

Ao final gere um arquivo no formato csv e um outro arquivo no formato parquet chamado (problema1_normalizado), eles serão destinados para pessoas distintas.

*EXTRA:* executar as mesmas validações no *1E8.csv.gz

In [89]:
#ler arquivo


spark = SparkSession.builder.appName("Spark-Ingest-dados_cadastrais").getOrCreate()


df_dados_cadastrais = spark.read.csv ("/content/drive/MyDrive/dados_cadastrais_fake.csv",header=True,sep =';') 




In [102]:
#df_dados_cadastrais.count()
df_dados_cadastrais.select('*').show()
 

+------------------+-----+-------------------+----------------+--------------+------------------+
|             nomes|idade|             cidade|          estado|           cpf|              cnpj|
+------------------+-----+-------------------+----------------+--------------+------------------+
|    Dennis Daniels|   31|         ACRELÂNDIA|              AC|   97566536800|    06589184909526|
|       Leah Becker|   42|        ÁGUA BRANCA|              AL|425.263.807-07|25.673.336/2350-20|
|        Sally Ford|   18|           ALVARÃES|              AM|   34647754103|    26543101702989|
|    Colleen Duncan|   21|     SERRA DO NAVIO|              AP|252.531.560-03|19.062.080/5100-98|
|   Jeff Stephenson|   73|             ABAÍRA|              BA|   49668886542|    97794530015384|
|     Sydney Curtis|   85|            ABAIARA|              CE|506.202.907-49|29.476.298/0856-78|
|    Kelly Matthews|   44|           Brasília|distrito federal|   39154836808|    24709301957761|
|         Juan Ruiz|

In [122]:


df = df_dados_cadastrais.withColumn('cpf',regexp_replace('cpf', '-',''))
  

#df.withColumn('address', regexp_replace('address', 'Rd', 'Road')) \  .show(truncate=False)



In [123]:
df.select('*').show()

+------------------+-----+-------------------+----------------+-------------+------------------+
|             nomes|idade|             cidade|          estado|          cpf|              cnpj|
+------------------+-----+-------------------+----------------+-------------+------------------+
|    Dennis Daniels|   31|         ACRELÂNDIA|              AC|  97566536800|    06589184909526|
|       Leah Becker|   42|        ÁGUA BRANCA|              AL|425.263.80707|25.673.336/2350-20|
|        Sally Ford|   18|           ALVARÃES|              AM|  34647754103|    26543101702989|
|    Colleen Duncan|   21|     SERRA DO NAVIO|              AP|252.531.56003|19.062.080/5100-98|
|   Jeff Stephenson|   73|             ABAÍRA|              BA|  49668886542|    97794530015384|
|     Sydney Curtis|   85|            ABAIARA|              CE|506.202.90749|29.476.298/0856-78|
|    Kelly Matthews|   44|           Brasília|distrito federal|  39154836808|    24709301957761|
|         Juan Ruiz|   39|    

**Problema 2**: Você deverá implementar um programa, para ler, tratar e particionar os dados.

O arquivo fonte está disponível em `https://st-it-cloud-public.s3.amazonaws.com/people-v2_1E6.csv.gz`

### Data Quality

- Higienizar e homogenizar o formato da coluna `document`
- Detectar através da coluna `document` se o registro é de uma Pessoa Física ou Pessoa Jurídica, adicionando uma coluna com essa informação
- Higienizar e homogenizar o formato da coluna `birthDate`
- Existem duas colunas nesse dataset que em alguns registros estão trocadas. Quais são essas colunas? 
- Corrigir os dados com as colunas trocadas
- Além desses pontos, existem outras tratamentos para homogenizar esse dataset. Aplique todos que conseguir.

### Agregação dos dados

- Quais são as 5 PF que mais gastaram (`totalSpent`)? 
- Qual é o valor de gasto médio por estado (`state`)?
- Qual é o valor de gasto médio por `jobArea`?
- Qual é a PF que gastou menos (`totalSpent`)?
- Quantos nomes e documentos repetidos existem nesse dataset?
- Quantas linhas existem nesse dataset?

### Particionamento de dados tratados com as regras descritas em `DATA QUALITY`

- Particionar em arquivos PARQUET por estado (`state`)
- Particionar em arquivos CSV por ano/mes/dia de nascimento (`birthDate`)